# Interacting with the IBM Watson Natural Language Understanding API; POST vs GET

Another useful API, especially when dealing with text, is the [IBM Watson  Natural Language Understanding API](https://console.bluemix.net/catalog/services/natural-language-understanding), which offers a variety of text analysis functionalities, such as sentiment analysis, entity extraction, keyword extraction, etc.

We will give a couple of examples below, to understand how we can take an unstructured piece of text (either the text alone, or a URL with text), and extract some "semi-structured" representation of its content.



## /analyze call

We will first start with the `GET /analyze` API call ([documentation](https://www.ibm.com/watson/developercloud/natural-language-understanding/api/v1/#get-analyze)), which takes as input a piece of text, and returns an analysis across various dimensions.

The call below gets as input a "text" variable, and returns back the sentiment of the text.

In [1]:
import requests
import json

def getSentiment(text):
    endpoint = "https://gateway.watsonplatform.net/natural-language-understanding/api/v1/analyze"

    # You can register and get your own credentials
    # The ones below have a quota of 1000 calls per day 
    # and can run out quickly if multiple people use these
    username = "711b4792-170d-490f-ac0e-5d785a271868"
    password = "No7JUOE1UOYu"
    
    parameters = {
        'features': 'emotion,sentiment',
        'version' : '2017-02-27',
        'text': text,
        'language' : 'en',
        # url = url_to_analyze, this is an alternative to sending the text
    }

    resp = requests.get(endpoint, params=parameters, auth=(username, password))
    
    return resp.json()

In [10]:
# Some text from http://www.espn.com/nba/story/_/id/26833423/giannis-wants-bucks-free-agents-back-next-year

# We will analyze the text below using the IBM Watson API

text = '''
There are certain people you meet throughout your life that make a (positive) difference. Chris is definitely one of them. Since I moved to the U.S. he has been one of my closest and dearest friends. I think he is an admirable and exemplary person in every possible aspect, on top of being really fun and sharing all the “Aussie traits” mentioned earlier. Yes, that’s right! He is an Aussie and he loves to travel!

So who would be best to talk about Australia than one of its expat locals, who has lived in Perth and Sydney and has traveled the country upside down?

I hope you enjoy this post. I couldn’t thank Chris enough for devoting his time to preparing such a great amount of valuable information. If I had the power to send everybody that reads this a treasure, it would be a friend like Chris.
'''

In [11]:
data = getSentiment(text)

Now, let's try to understand the structure of the answer. First, we check the high-level keys.

In [12]:
data.keys()

dict_keys(['usage', 'sentiment', 'language', 'emotion'])

Now, let's check the content of these keys:

In [13]:
data['language']

'en'

In [14]:
data['sentiment']

{'document': {'score': 0.880418, 'label': 'positive'}}

In [15]:
data['emotion']

{'document': {'emotion': {'sadness': 0.472408,
   'joy': 0.700614,
   'fear': 0.049824,
   'disgust': 0.05452,
   'anger': 0.070944}}}

In [16]:
# Let's go deeper into the 'emotion' dictionary
data['emotion']['document']

{'emotion': {'sadness': 0.472408,
  'joy': 0.700614,
  'fear': 0.049824,
  'disgust': 0.05452,
  'anger': 0.070944}}

In [17]:
# And a bit more
data['emotion']['document']['emotion']

{'sadness': 0.472408,
 'joy': 0.700614,
 'fear': 0.049824,
 'disgust': 0.05452,
 'anger': 0.070944}

### Exercise

Type your own piece of text, and analyze it to extract sentiment and emotions. Discuss your findings

## Entities call

[Full Documentation of the call](https://www.ibm.com/watson/developercloud/natural-language-understanding/api/v1/#entities)

This is a an API call that extracts entities from the text, and also the sentiment and emotion for each of these entities.

There are two new technical aspects with this API. First of all, we use the command `requests.post` as opposed to `requests.get`. This happens because `GET` is designed to handle limited amount of data. When we have a large volume of data to send as parameters, then the HTTP protocol requires the use of the `POST` command. You will also see that the parameters that we pass are not "flat" as they used to be. Instead we submit the `watson_options` as the set of parameters, which is itself semi-structured.

In terms of natural language processing, we will examine a couple of capabilities of the API. First, you will see that there is the capability of "normalizing" each entity, so that two different ways of saying the same thing get mapped to the same entity. So for example, "President Trump" and "Donald Trump" get mapped to the same Knowledge Graph entity.

In [18]:
import requests
import json

def processURL(url_to_analyze):
    endpoint_watson = "https://gateway.watsonplatform.net/natural-language-understanding/api/v1/analyze"
    params = {
        'version': '2017-02-27',
    }
    headers = { 
        'Content-Type': 'application/json',
    }
    watson_options = {
      "url": url_to_analyze,
      "features": {
        "entities": {
          "sentiment": True,
          "emotion": True,
          "limit": 10
        }
      }
    }
    username = "711b4792-170d-490f-ac0e-5d785a271868"
    password = "No7JUOE1UOYu"

    resp = requests.post(endpoint_watson, 
                         data=json.dumps(watson_options), 
                         headers=headers, 
                         params=params, 
                         auth=(username, password) 
                        )
    return resp.json()

In [19]:
url_to_analyze = 'https://www.aljazeera.com/news/2019/05/iran-hits-trump-nuclear-weapons-comment-190527234526280.html'

data = processURL(url_to_analyze)

In [20]:
# Let's see what we get back as top-level attributes
data.keys()

dict_keys(['usage', 'retrieved_url', 'language', 'entities'])

In [21]:
# Let' see the entities list
data["entities"]

[{'type': 'Location',
  'text': 'Iran',
  'sentiment': {'score': -0.469293, 'label': 'negative'},
  'relevance': 0.938809,
  'emotion': {'sadness': 0.211997,
   'joy': 0.161907,
   'fear': 0.149669,
   'disgust': 0.111536,
   'anger': 0.522634},
  'disambiguation': {'subtype': ['Country'],
   'name': 'Iran',
   'dbpedia_resource': 'http://dbpedia.org/resource/Iran'},
  'count': 15},
 {'type': 'Person',
  'text': 'President Donald Trump',
  'sentiment': {'score': -0.013948, 'label': 'negative'},
  'relevance': 0.861917,
  'emotion': {'sadness': 0.520429,
   'joy': 0.136396,
   'fear': 0.12346,
   'disgust': 0.15311,
   'anger': 0.11295},
  'disambiguation': {'subtype': ['AwardNominee',
    'AwardWinner',
    'Celebrity',
    'CompanyFounder',
    'TVPersonality',
    'TVProducer',
    'FilmActor',
    'TVActor'],
   'name': 'Donald Trump',
   'dbpedia_resource': 'http://dbpedia.org/resource/Donald_Trump'},
  'count': 11},
 {'type': 'Location',
  'text': 'United States',
  'sentiment': {

In [22]:
# Let' see the first entity. Notice the "disambiguated" attribute that
# points to "canonical" versions of the entity, in DBPedia, Freebase, OpenCYC, YAGO, etc
data["entities"][0]

{'type': 'Location',
 'text': 'Iran',
 'sentiment': {'score': -0.469293, 'label': 'negative'},
 'relevance': 0.938809,
 'emotion': {'sadness': 0.211997,
  'joy': 0.161907,
  'fear': 0.149669,
  'disgust': 0.111536,
  'anger': 0.522634},
 'disambiguation': {'subtype': ['Country'],
  'name': 'Iran',
  'dbpedia_resource': 'http://dbpedia.org/resource/Iran'},
 'count': 15}

In [23]:
# This function takes as input the result
# from the IBM Watson API and returns a list
# of entities that are relevant (above threshold)
# to the article
def getEntities(data, threshold):
    result = []
    for entity in data["entities"]:
        relevance = float(entity['relevance'])
        if relevance > threshold:
            result.append(entity['text'])
    return result

getEntities(data, 0.25)

['Iran',
 'President Donald Trump',
 'United States',
 'Mohammad Javad Zarif',
 'Tehran',
 'Trump White House',
 'Middle East',
 'Japan',
 'Iranian President Hassan Rouhani']

### Exercise

* First of all, **get your own credentials for the IBM Watson API**. The demo key that we use above has a limited quota.
* Use an API to get news articles. 
    * Option 1: Use the API at https://newsapi.org to fetch the news from various sources. Print the entities that are currently being discussed in the news, together with their relevance value and the associated sentiment.
    * Option 2: Use the NY Times API to fetch the Top Stories News. You can register and get an API key at https://developer.nytimes.com/. The `Top Stories V2 API` provides the details of the news of the day: (The API call documentation is at https://developer.nytimes.com/top_stories_v2.json and the API Call is  https://api.nytimes.com/svc/topstories/v2/home.json?api-key=PUTYOURKEYHERE). Repeat the entity extraction process from above.
    * Option 3: Use the Guardian API at https://open-platform.theguardian.com/documentation/ to fetch news from The Guardian.
